In [7]:
#import minsearch
from groq import Groq
import json
import requests
import os


In [8]:
client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

In [9]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [10]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [ ]:
# index = minsearch.Index(
#     text_fields=["question", "text", "section"],
#     keyword_fields=["course"]
# )

SELECT * WHERE course = 'data-engineering-zoomcamp';

In [ ]:
q = 'the course has already started, can I still enroll?'

In [ ]:
#index.fit(documents)

In [10]:
# api_key = os.environ.get("GROQ_API_KEY")
# url = "https://api.groq.com/openai/v1/models"

# headers = {
#     "Authorization": f"Bearer {api_key}",
#     "Content-Type": "application/json"
# }

# response = requests.get(url, headers=headers)

# # Pretty-print the JSON response
# print(json.dumps(response.json(), indent=2))

In [ ]:
# def search(query):
#     boost = {'question': 3.0, 'section': 0.5}

#     results = index.search(
#         query=query,
#         filter_dict={'course': 'data-engineering-zoomcamp'},
#         boost_dict=boost,
#         num_results=5
#     )

#     return results

In [13]:
# prompt_template = """
# You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
# Use only the facts from the CONTEXT when answering the QUESTION.

# QUESTION: {question}

# CONTEXT: 
# {context}
# """.strip()

In [14]:
# context = ""
    
# for doc in results:
#     context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
# print(context)

In [15]:
# prompt = prompt_template.format(question=q, context=context).strip()

In [16]:
# from groq import Groq

# client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

# response = client.chat.completions.create(
#     model="llama-3.3-70b-versatile",  
#     messages=[{"role": "user", "content": prompt}],
#     max_tokens=1000,
# )

# print(response.choices[0].message.content)

In [17]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [18]:
# def llm(prompt):
#     response = client.chat.completions.create(
#         model='gpt-4o',
#         messages=[{"role": "user", "content": prompt}]
#     )
    
#     return response.choices[0].message.content

In [18]:
def llm(prompt):

    response = client.chat.completions.create(
        model="llama-3.3-70b-versatile",  
        messages=[{"role": "user", "content": prompt}],
        max_tokens=1000,
    )
    
    return response.choices[0].message.content

In [ ]:
#rag(query)

'To run Kafka, you can use the following methods:\n\n1. For Java Kafka: In the project directory, run `java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java` in the terminal.\n\n2. For Python Kafka: \n   - Create a virtual environment by running `python -m venv env` and then `source env/bin/activate` (on MacOS, Linux) or `env/Scripts/activate` (on Windows).\n   - Install the necessary packages by running `pip install -r ../requirements.txt` in the virtual environment.\n   - Ensure that Docker images are up and running before running the Python file.\n   - Alternatively, if you encounter a "ModuleNotFoundError: No module named \'kafka.vendor.six.moves\'" error, use `pip install kafka-python-ng` instead of the original Kafka package. \n\nNote: Make sure to replace `<jar_name>` with your actual jar file name when running the Java command.'

In [23]:
print(rag(query))

To run Kafka, you can follow these steps based on the provided context:

**For Java:**
1. Navigate to the project directory.
2. Run the command: `java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java`

**For Python:**
1. Create a virtual environment by running: `python -m venv env`
2. Activate the virtual environment: 
   - On MacOS and Linux, run `source env/bin/activate`
   - On Windows, run `env\Scripts\activate`
3. Install the required packages: `pip install -r ../requirements.txt`
4. Ensure you have the necessary dependencies installed, such as `kafka-python-ng` to avoid module errors: `pip install kafka-python-ng`

Note: Make sure Docker images are up and running before attempting to run Python files related to Kafka. Also, be mindful of file permissions and adjust them as necessary (e.g., using `chmod +x build.sh` if you encounter a "Permission denied" error).


In [24]:
rag('the course has already started, can I still enroll?')

"Yes, even if you don't register, you're still eligible to submit the homeworks. However, be aware that there will be deadlines for turning in the final projects, so don't leave everything for the last minute."

In [1]:
from elasticsearch import Elasticsearch

In [2]:
es_client = Elasticsearch('http://localhost:9200') 

In [4]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [9]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [5]:
from tqdm.auto import tqdm

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

100%|██████████| 948/948 [00:04<00:00, 235.75it/s]


In [25]:
query = 'how do I access the course materials?'

In [13]:
search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }


In [14]:
response = es_client.search(index=index_name, body=search_query)

In [16]:
result_docs = []

for hit in response['hits']['hits']:
    result_docs.append(hit['_source'])

print(result_docs[0:2])

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.", 'section': 'General course-related questions', 'question': 'Course - Can I still join the course after the start date?', 'course': 'data-engineering-zoomcamp'}, {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.', 'section': 'General course-related questions', 'question': 'Course - Can I follow the course after it finishes?', 'course': 'data-engineering-zoomcamp'}]


In [19]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [20]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [26]:
print(rag(query))

To access the course materials, you should clone the course repository to your local machine using Git. This is explained in the video "Git for Everybody: How to Clone a Repository from GitHub". Having the local repository will allow you to access the instructors' code and other course content.
